In [1]:
#!pip install Keras==2.2.4
#from importlib import reload  
import autoreload
%load_ext autoreload
%autoreload 2
#import loader
#reload(loader)
#from loader.fb_image_gen import *

In [2]:
%tensorflow_version 1.14
import keras
import keras_preprocessing


`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


Using TensorFlow backend.


In [3]:
#@title Default title text
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
if os.path.exists('/content/data/query.zip'):
  print("query.zip already exists; skip download and unzip")
else:
  print("starting to copy and unzip")
  !mkdir ./data
  !cp /content/drive/MyDrive/collab_drive/reference.zip /content/data/reference.zip
  !cp /content/drive/MyDrive/collab_drive/query.zip /content/data/query.zip
  !unzip -qq data/query.zip
  !unzip -qq data/reference.zip


starting to copy and unzip


In [5]:
!cp -a /content/drive/MyDrive/collab_drive/isc2021/. /content/

In [6]:
#import keras
#keras.applications.resnet50.preprocess_input
!cp /content/drive/MyDrive/collab_drive/isc2021/utils.py /content/

!cp /content/drive/MyDrive/collab_drive/isc2021/triplet_hardlib.py /content/

In [7]:
#!ls
#!pwd

Now upload the file from local file system to runtime

In [8]:
#from google.colab import files
#uploaded = files.upload()
#for fn in uploaded.keys():
#  print('User uploaded file "{name}" with length {length} bytes'.format(
#      name=fn, length=len(uploaded[fn])))


**Import files and libraries**

In [9]:
''' Simple triplet based similarity training;
Simple: using a simple model with dense layers and few conv2d only
ref:https://github.com/Ekeany/Siamese-Network-with-Triplet-Loss/blob/master/MachinePart1.ipynb
#requires keras 2.2.5(cityscape env)
'''
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#from args import get_arguments
import argparse
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from pathlib import Path
from settings import * # importing all the variables and Cosntants
#from loader.fb_image_gen import *
#from models.resnet_50good import *
from models.resnet50Reg import *
from loader.fb_image_gen_pre import *
from keras.callbacks import EarlyStopping,ModelCheckpoint, TensorBoard

#from utils import *
#from train_hard import *

import time
from datetime import datetime
!pip install faiss



  found 4991 images
  found 4991 images
     |████████████████████████████████| 4.7 MB 7.3 MB/s 


In [10]:
!apt-get install libomp-dev
import faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 40 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (201 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 148489 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Se

In [11]:
  
import autoreload
%load_ext autoreload
%autoreload 2
from utils import *

from importlib import reload  
from triplet_hardlib import *
from models.resnet50Reg import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
#from importlib import reload  
#import models
#reload(models.resnet_50good)
#import loader
#reload(loader.fb_image_gen)
#from models.resnet_nn_drop import *
#print(get_model_name())


In [17]:
#from keras.optimizers  import Adam
#keras.optimizers.Adam
base_model = embedding_model()
testgen = generate_triplets(BATCH_SIZE=16)
data = next(testgen )
[A, P, N], label = data
xq = np.squeeze(base_model.predict(A))
xb = np.squeeze(base_model.predict(P))
print(xq.shape)
d=256#dimension of embeddings
index = faiss.IndexFlatL2(d)
index.add(xb)
D, I = index.search(xq, 2) # search top 2
print("matching index before training....")
print(I)
print(D)
getMatchingScore(I, 1)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/tensorflow-1.15.2/python3.7/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step

(16, 256)
matching index before training....
[[ 0  4]
 [10  9]
 [ 2  1]
 [ 3  1]
 [ 4  9]
 [ 5  2]
 [ 6 15]
 [ 7  0]
 [ 8 13]
 [ 4  9]
 [ 1 10]
 [11 13]
 [ 1 11]
 [13  1]
 [ 1  4]
 [13 10]]
[[0.56678224 0.84736836]
 [1.0135784  1.0509391 ]
 [0.77452135 1.170873  ]
 [0.714881   0.9821086 ]
 [0.7936989  0.9613153 ]
 [1.1205447  1.122436  ]
 [0.76356304 0.9197724 ]
 [0.7608577  0.9980471 ]
 [1.0701119  1.1474273 ]
 [1.0559077  1.0829166 ]
 [0.9263317  1.0909364 ]
 [0.9053847  1.1083851 ]
 [0.98948866 1.1239326 ]
 [0.5890514  1.0122061 ]
 [1.1306446  1.1539698 ]
 [0.9279753  1.0189602 ]]
Top K=1 matchign accuracy 62.5% : 


In [23]:

base_model = embedding_model()
model = complete_model(base_model)
model.summary()
model.compile(loss=identity_loss, optimizer=Adam(1e-4))
modelFilePath = "./models/weights/"
model, base_model = train_basic(model, base_model, epochs=50)
I_1 = findAccuracy(base_model)
pickle.dump(I_1, open("L2Index_2_prev.p", "wb"))
model, base_model = test_hardOfflineBatch(model, base_model, epochs=30)
#model.save_weights(modelFilePath+"complete_res50_offline.hdf5")
#base_model.save_weights(modelFilePath + "Embeddings_res50_off_1.hdf5")
Ioffline = findAccuracy(base_model)

/tensorflow-1.15.2/python3.7/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Embedding (Model)               (None, 256)          36761984    input_3[0][0]                    
                                                                 input_4[0][0]              

100%|██████████| 155/155 [00:44<00:00,  3.50it/s]


Time for generation 0.7380481123924255 mins
skipping writing embeddings to file
Inference for generating embedding. THis will take long....


100%|██████████| 155/155 [00:56<00:00,  2.74it/s]


...

Time for generation 0.9452891667683919 mins
skipping writing embeddings to file
matching index after training....
Top K=1 matchign accuracy 78.04435483870968% : 
I[0:10 [[   0]
 [3824]
 [   2]
 [1600]
 [   4]
 [   5]
 [   6]
 [   7]
 [   8]
 [   9]]
D[0:10 [[0.36924028]
 [0.51550233]
 [0.5909394 ]
 [0.4030758 ]
 [0.45681822]
 [0.35046935]
 [0.5698638 ]
 [0.4848777 ]
 [0.6344483 ]
 [0.45287406]]
Inference for generating embedding. THis will take long....


100%|██████████| 155/155 [00:44<00:00,  3.48it/s]


Time for generation 0.7434163729349772 mins
skipping writing embeddings to file
Inference for generating embedding. THis will take long....


100%|██████████| 155/155 [00:56<00:00,  2.73it/s]


Time for generation 0.9472572763760885 mins
skipping writing embeddings to file
matching index after training....
Top K=1 matchign accuracy 78.04435483870968% : 
I[0:10 [[   0]
 [3824]
 [   2]
 [1600]
 [   4]
 [   5]
 [   6]
 [   7]
 [   8]
 [   9]]
D[0:10 [[0.36924028]
 [0.51550233]
 [0.5909394 ]
 [0.4030758 ]
 [0.45681822]
 [0.35046935]
 [0.5698638 ]
 [0.4848777 ]
 [0.6344483 ]
 [0.45287406]]


In [24]:
from utils import *
#Ioffline = findAccuracy(base_model)
print(Ioffline[0:5])

[[   0]
 [3824]
 [   2]
 [1600]
 [   4]]


In [ ]:
#save these weiths, these are best we got for now
#model.save_weights(modelFilePath+"complete_res50Reg.hdf5")
#base_model.save_weights(modelFilePath + "Embeddings_res50Reg.hdf5")

In [ ]:
#pickle.load(open("./L2Index_2_prev.p", "rb"))
#save it in filename that is read as hardocded by offline
#pickle.dump(Ioffline, open("L2Index_2_prev", "wb"))

In [25]:

model, base_model = test_hardbatch(model, base_model, epochs=20)
Ihardbatch = findAccuracy(base_model)

model weights filepath name is:  ./models/weights/model_resnet50Reg_EP20_26-08-2021_H23_M37.hdf5
Starting training process!
-------------------------------------
100/2480 -------------
[100] Time for 100 iterations: 3.4 mins, Train Loss: 0.04216323047876358
val_loss =  0.083433
best loss found, previous: 1000, current: 0.08343300223350525 
200/2480 -------------
[200] Time for 200 iterations: 7.2 mins, Train Loss: 0.041830845177173615
val_loss =  0.09209647
300/2480 -------------
[300] Time for 300 iterations: 11.1 mins, Train Loss: 0.028824439272284508
val_loss =  0.08912809
400/2480 -------------
[400] Time for 400 iterations: 14.9 mins, Train Loss: 0.04988332465291023
val_loss =  0.09876922
500/2480 -------------
[500] Time for 500 iterations: 18.7 mins, Train Loss: 0.03111330047249794
val_loss =  0.07895168
best loss found, previous: 0.08343300223350525, current: 0.07895167917013168 
600/2480 -------------
[600] Time for 600 iterations: 22.5 mins, Train Loss: 0.02925398200750351
va

100%|██████████| 155/155 [00:43<00:00,  3.54it/s]


Time for generation 0.7298391501108805 mins
skipping writing embeddings to file
Inference for generating embedding. THis will take long....


100%|██████████| 155/155 [00:55<00:00,  2.77it/s]


Time for generation 0.9327014684677124 mins
skipping writing embeddings to file
matching index after training....
Top K=1 matchign accuracy 78.08467741935485% : 
I[0:10 [[1757]
 [3824]
 [   2]
 [   3]
 [   4]
 [ 431]
 [   6]
 [   7]
 [   8]
 [   9]]
D[0:10 [[0.37015486]
 [0.43849158]
 [0.35956848]
 [0.17071271]
 [0.61112666]
 [0.31321847]
 [0.40811896]
 [0.28070283]
 [0.5024842 ]
 [0.2719289 ]]


In [26]:
pickle.dump(Ihardbatch, open("L2Index_2_prev.p", "wb"))
#pickle.dump(Ioffline, open("Ioffline.p", "wb"))
#pickle.dump(Ihardbatch, open("Ihardbatch.p", "wb"))

In [27]:

model, base_model = test_hardOfflineBatch(model, base_model, epochs=25)
Ifinal = findAccuracy(base_model)

#pickle.dump(Ifinal, open("Ihardbatch.p", "wb"))

train_stop=  869
Starting training process!
-------------------------------------
13/325 -------------
[13] Time for 13 iterations: 0.4 mins, Train Loss: 0.1794608235359192
val_loss =  0.55272067
best loss found, previous: 1000, current: 0.5527206659317017 
curr best_val_index=  13
26/325 -------------
[26] Time for 26 iterations: 1.0 mins, Train Loss: 0.1545369178056717
val_loss =  0.39752883
best loss found, previous: 0.5527206659317017, current: 0.39752882719039917 
curr best_val_index=  26
39/325 -------------
[39] Time for 39 iterations: 1.6 mins, Train Loss: 0.07824991643428802
val_loss =  0.3611556
best loss found, previous: 0.39752882719039917, current: 0.36115559935569763 
curr best_val_index=  39
52/325 -------------
[52] Time for 52 iterations: 2.1 mins, Train Loss: 0.06513358652591705
val_loss =  0.29272
best loss found, previous: 0.36115559935569763, current: 0.2927199900150299 
curr best_val_index=  52
65/325 -------------
[65] Time for 65 iterations: 2.7 mins, Train Loss

100%|██████████| 155/155 [00:44<00:00,  3.51it/s]


Time for generation 0.7355733553568522 mins
skipping writing embeddings to file
Inference for generating embedding. THis will take long....


100%|██████████| 155/155 [00:56<00:00,  2.76it/s]


Time for generation 0.9364423831303914 mins
skipping writing embeddings to file
matching index after training....
Top K=1 matchign accuracy 88.4475806451613% : 
I[0:10 [[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]]
D[0:10 [[0.51307297]
 [0.66773164]
 [0.38707614]
 [0.33157837]
 [0.58784795]
 [0.3767109 ]
 [0.54260397]
 [0.37214792]
 [0.5883399 ]
 [0.31562376]]
Inference for generating embedding. THis will take long....


100%|██████████| 155/155 [00:43<00:00,  3.52it/s]


Time for generation 0.73314395348231 mins
skipping writing embeddings to file
Inference for generating embedding. THis will take long....


100%|██████████| 155/155 [00:56<00:00,  2.76it/s]


Time for generation 0.9370714942614238 mins
skipping writing embeddings to file
matching index after training....
Top K=1 matchign accuracy 88.4475806451613% : 
I[0:10 [[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]]
D[0:10 [[0.51307297]
 [0.66773164]
 [0.38707614]
 [0.33157837]
 [0.58784795]
 [0.3767109 ]
 [0.54260397]
 [0.37214792]
 [0.5883399 ]
 [0.31562376]]


In [30]:
pickle.dump(Ifinal, open("L2Index_2_prev.p", "wb"))
model, base_model = test_hardOfflineBatch(model, base_model, epochs=50)

train_stop=  458
Starting training process!
-------------------------------------
7/350 -------------
[7] Time for 7 iterations: 0.2 mins, Train Loss: 0.0119495689868927
val_loss =  0.16407399
best loss found, previous: 1000, current: 0.16407398879528046 
curr best_val_index=  7
14/350 -------------
[14] Time for 14 iterations: 0.6 mins, Train Loss: 0.012646839953958988
val_loss =  0.15562354
best loss found, previous: 0.16407398879528046, current: 0.15562354028224945 
curr best_val_index=  14
21/350 -------------
[21] Time for 21 iterations: 0.9 mins, Train Loss: 0.010720161721110344
val_loss =  0.17263865
28/350 -------------
[28] Time for 28 iterations: 1.2 mins, Train Loss: 0.013329533860087395
val_loss =  0.18238015
35/350 -------------
[35] Time for 35 iterations: 1.5 mins, Train Loss: 0.012284592725336552
val_loss =  0.15439734
best loss found, previous: 0.15562354028224945, current: 0.15439733862876892 
curr best_val_index=  35
42/350 -------------
[42] Time for 42 iterations: 

100%|██████████| 155/155 [00:44<00:00,  3.51it/s]


Time for generation 0.7361020008722942 mins
skipping writing embeddings to file
Inference for generating embedding. THis will take long....


100%|██████████| 155/155 [00:56<00:00,  2.76it/s]


Time for generation 0.9368921637535095 mins
skipping writing embeddings to file
matching index after training....
Top K=1 matchign accuracy 90.7258064516129% : 
I[0:10 [[   0]
 [   1]
 [   2]
 [1415]
 [   4]
 [   5]
 [   6]
 [ 500]
 [   8]
 [   9]]
D[0:10 [[0.5852598 ]
 [0.61758673]
 [0.43868756]
 [0.32360303]
 [0.6827091 ]
 [0.51620317]
 [0.66561735]
 [0.4507252 ]
 [0.71506095]
 [0.2454356 ]]


In [29]:

model.save_weights(modelFilePath +get_model_name()+ "complete_90%.hdf5")
base_model.save_weights(modelFilePath + get_model_name()+"base_90%.hdf5")
!cp ./models/weights/*.hdf5 /content/drive/MyDrive/collab_drive/model_artifacts

In [31]:
#!cp -a ./models/weights/ /content/drive/MyDrive/collab_drive/model_artifacts
#!cp -a ./models/logs/ /content/drive/MyDrive/collab_drive/model_artifacts

!cp /content/*.py /content/drive/MyDrive/collab_drive/isc2021/
!cp /content/models/*.py /content/drive/MyDrive/collab_drive/isc2021/models/
!cp /content/loader/*.py /content/drive/MyDrive/collab_drive/isc2021/loader/
#!cp /content/*.p /content/drive/MyDrive/collab_drive/model_artifacts/

In [ ]:
#!cp /content/models/*.py /content/drive/MyDrive/collab_drive/isc2021/models/
#!cp /content/loader/*.py /content/drive/MyDrive/collab_drive/isc2021/loader/
#!cp /content/*.ipynb /content/drive/MyDrive/collab_drive/isc2021/

In [ ]:
'''
XQ = gen_embeddings(base_model=base_model,file_list=Q_List,file_ids=Q_IDS,outFile='./models/weights/subset_query_resnet50.hdf5')
#XQ = gen_embeddings(base_model=base_model,file_list=Q_List,file_ids=Q_IDS,outFile='')
XD = gen_embeddings(base_model=base_model,file_list=R_List,file_ids=REF_IDS,outFile='./models/weights/subset_ref_resnet50.hdf5')
#XD = gen_embeddings(base_model=base_model,file_list=R_List,file_ids=REF_IDS,outFile='')
print(XQ.shape)
print(XD.shape)
'''

"\nXQ = gen_embeddings(base_model=base_model,file_list=Q_List,file_ids=Q_IDS,outFile='./models/weights/subset_query_resnet50.hdf5')\n#XQ = gen_embeddings(base_model=base_model,file_list=Q_List,file_ids=Q_IDS,outFile='')\nXD = gen_embeddings(base_model=base_model,file_list=R_List,file_ids=REF_IDS,outFile='./models/weights/subset_ref_resnet50.hdf5')\n#XD = gen_embeddings(base_model=base_model,file_list=R_List,file_ids=REF_IDS,outFile='')\nprint(XQ.shape)\nprint(XD.shape)\n"